In [1]:
%load_ext autoreload
%autoreload 2

import sys
import cv2
import numpy as np
sys.path.insert(1, '../../Src/')
from utils import vision
from utils import camera
import time
import traceback

COLOR_BOUNDINGBOX = (255, 0, 0)
THICKNESS_BOUNDINGBOX = 2


In [2]:
from models.faceboxes.faceboxes_tensorflow import FaceBoxes_tensorflow

2023-05-28 18:33:31.803553: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-28 18:33:32.823372: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Instructions for updating:
non-resource variables are not supported in the long term


In [10]:
face_boxes = FaceBoxes_tensorflow.FaceDetector("FaceBoxesProd.pb",device='gpu',gpu_memory_fraction=0.1)

2023-05-28 18:34:27.375575: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-28 18:34:27.375829: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-28 18:34:27.375982: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [11]:
cap = camera.ParallelCamera(0).start()

start webcam


In [12]:


my_fps = vision.FPS(100)

my_fps_model = vision.FPS(100)


while True:
  my_fps.start()

  # Read image
  success, image = cap.read()  
  # Flip the image horizontally for a selfie-view display.    
  image = cv2.flip(image.copy(), 1)

  image = cv2.resize(image,(300,300))
  #time.sleep(0.5)

  if not success:
    print("Ignoring empty camera frame.")
    continue

  my_fps_model.start()
  
  # Model Inference
  image.flags.writeable = False
  #predictions = model.predict(image)
  boxes, scores= face_boxes.predict(image)
  image.flags.writeable = True  
  
  # Draw bounding boxes
  image = vision.draw_boundingboxes(image,boxes,COLOR_BOUNDINGBOX,THICKNESS_BOUNDINGBOX,if_wh=False)
  
  # Calculate FPS
  my_fps_model.update()
  my_fps.update()
  # Show image
  image = vision.put_text(image,"FPS      : "+str(np.round(my_fps.fps(),3)),pos =(20,20))
  image = vision.put_text(image,"FPS model: "+str(np.round(my_fps_model.fps(),3)),pos =(20,50))
  cv2.imshow('Face Detection Project',image)
 
  # Close window with ESC
  if cv2.waitKey(5) & 0xFF == 27:
    break

# DestroyWindows
cap.release()
cv2.destroyAllWindows()


In [6]:
cap.release()